# Convert a host-guest pair from `https://github.com/MobleyLab/benchmarksets/`

The basic strategy is going to be:
0. Download the host-guest file from GitHub, so we know we are starting fresh.
1. Create a PDB with proper `CONECT` records.
2. Split the PDB into separate topologies, starting by extracting the host and guest.
3. Use the host and guest topologies to create `mol2` files (with SYBYL atom types) for the host and guest.
4. Extract the dummy atoms, water, and ions into a separate PDB -- we'll come back to these later.
4. Read the host and guest `mol2` files into a list of `OEMol`s.
5. Then, use the list of `OEMol`s to create an OpenMM system.
6. Convert the OpenMM system into a ParmEd structure, so we can merge the host and guest with water and ions.

In [87]:
%load_ext autoreload
%autoreload 2

import os as os
import urllib.request

import parmed as pmd

from openforcefield.typing.engines.smirnoff import ForceField, unit
from openforcefield.utils import mergeStructure

from utils import (create_pdb_with_conect, prune_conect)
from utils import (split_topology, create_host_guest_topology)
from utils import (create_host_mol2, convert_mol2_to_sybyl_antechamber)
from utils import (load_mol2, check_unique_atom_names)
from utils import (check_bond_lengths)
from utils import (extract_water_and_ions, create_water_and_ions_parameters)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Step 1: process the input files

First, let's specify the input data.

In [61]:
test_case = 'cb7-1/'
destination = './tests/' + test_case
host_resname = 'CB7'
guest_resname = 'MOL'

Next, let's specify the output and intermediate files.

First, let's download a fresh host-guest example from David Mobley's `benchmarksets` GitHub repository.

In [76]:
if not os.path.exists(destination):
    os.makedirs(destination)
    
request = urllib.request.urlretrieve("https://rawgit.com/MobleyLab/benchmarksets/master/input_files/cb7-set1/prmtop-rst7/cb7-1.prmtop", destination + "cb7-1.prmtop")
request = urllib.request.urlretrieve("https://rawgit.com/MobleyLab/benchmarksets/master/input_files/cb7-set1/prmtop-rst7/cb7-1.rst7", destination + "cb7-1.rst7")

Next, we'll create a proper PDB from these AMBER files, and delete the `CONECT` records that are solvent-solvent.

In [63]:
create_pdb_with_conect(solvated_pdb=destination + 'cb7-1.rst7',
                      amber_prmtop=destination + 'cb7-1.prmtop',
                      output_pdb=destination + 'cb7-1.pdb')

In [64]:
prune_conect(input_pdb='cb7-1.pdb',
            output_pdb='cb7-1.pruned.pdb',
            path=destination)

We'll split the PDB into separate topology objects, and extract the host-guest topology for now.

In [65]:
components = split_topology(file_name=destination + 'cb7-1.pruned.pdb')
hg_topology = create_host_guest_topology(components, host_resname=host_resname, guest_resname=guest_resname)

We'll also need to create `mol2` files **with SYBYL atom types** for the host and guest, that we can use later.

In [75]:
create_host_mol2(
    solvated_pdb=destination + 'cb7-1.pruned.pdb',
    amber_prmtop=destination + 'cb7-1.prmtop',
    mask=host_resname,
    output_mol2=destination + host_resname + '.mol2')

create_host_mol2(
    solvated_pdb=destination + 'cb7-1.pruned.pdb',
    amber_prmtop=destination + 'cb7-1.prmtop',
    mask=guest_resname,
    output_mol2=destination + guest_resname + '.mol2')

convert_mol2_to_sybyl_antechamber(
    input_mol2=destination + host_resname + '.mol2',
    output_mol2=destination + host_resname + '-sybyl.mol2',
    ac_doctor=False)

convert_mol2_to_sybyl_antechamber(
    input_mol2=destination + guest_resname + '.mol2',
    output_mol2=destination + guest_resname + '-sybyl.mol2',
    ac_doctor=True)

We also need to separate out the dummy atoms, waters, and ions from the starting AMBER files.

In [81]:
extract_water_and_ions(
    amber_prmtop=destination + 'cb7-1.prmtop',
    amber_inpcrd=destination + 'cb7-1.rst7',
    host_residue=':' + host_resname,
    guest_residue=':' + guest_resname,
    dummy_atoms=True,
    output_pdb=destination + 'water_ions.pdb')

create_water_and_ions_parameters(
    input_pdb='water_ions.pdb',
    output_prmtop='water_ions.prmtop',
    output_inpcrd='water_ions.inpcrd',
    dummy_atoms=True,
    path=destination)

## Step 2: load in the files we just created

In [82]:
host = load_mol2(
    filename=destination + host_resname + '-sybyl.mol2',
    name=host_resname,
    add_tripos=True)

guest = load_mol2(
    filename=destination + guest_resname + '-sybyl.mol2',
    name=guest_resname,
    add_tripos=False)

check_unique_atom_names(host)
check_unique_atom_names(guest)
molecules = [host, guest]

## Create an OpenMM system

In [83]:
ff = ForceField('forcefield/smirnoff99Frosst.ffxml')
system = ff.createSystem(
    hg_topology.topology,
    molecules,
    nonbondedCutoff=1.1 * unit.nanometer,
    ewaldErrorTolerance=1e-4)

## Create a ParmEd system and check for bad bonds

In [84]:
hg_structure = pmd.openmm.topsystem.load_topology(
    hg_topology.topology, system, hg_topology.positions)

check_bond_lengths(hg_structure, threshold=4)

try:
    hg_structure.save(destination + 'hg.prmtop')
except OSError:
    print(
        'Check if the host-guest parameter file already exists...')

try:
    hg_structure.save(destination + 'hg.inpcrd')
except OSError:
    print(
        'Check if the host-guest coordinate file already exists...')

Check if the host-guest parameter file already exists...
Check if the host-guest coordinate file already exists...


## Load the water and ions into a ParmEd structure

In [85]:
water_and_ions = pmd.amber.AmberParm(
    destination + 'water_ions.prmtop',
    xyz=destination + 'water_ions.inpcrd')

Now, let's merge the host-guest structure with the dummy atoms, the waters, and the ions.

In [90]:
merged = mergeStructure(hg_structure, water_and_ions)

try:
    merged.save(destination_directory + destination_top)
except:
    print('Check if solvated parameter file already exists...')
try:
    merged.save(destination_directory + destination_crd)
except:
    print('Check if solvated coordinate file already exists...')


Check if solvated parameter file already exists...
Check if solvated coordinate file already exists...


Now, we should be done!

In [92]:
merged.box